In [1]:
import numpy as np
import pandas as pd
import matplotlib as matplot
import nltk
import sklearn as sk

# Question 1

In [2]:
en_df = pd.read_csv('data/CONcreTEXT_trial_EN.tsv', sep='\t') # load data files
it_df = pd.read_csv('data/CONcreTEXT_trial_IT.tsv', sep='\t')

In [3]:
df = pd.concat([en_df, it_df])
df = df.reset_index(0)
df

,index,TARGET,POS,INDEX,TEXT,MEAN
0,0,achievement,N,3,"Bring up academic achievements , awards , and ...",3.06
1,1,achievement,N,9,"Please list people you have helped , your pers...",3.03
2,2,activate,V,1,Add activated carbon straight to your vodka .,3.83
3,3,activate,V,15,"Place sensors around your garden , and when a ...",5.51
4,4,adventure,N,9,Look for a partner that shares your level of a...,2.03
...,...,...,...,...,...,...
195,95,verità,N,8,"In un modo o nell' altro , la verità viene sem...",2.53
196,96,viaggio,N,2,Organizza dei viaggi nel fine settimana quando...,5.03
197,97,viaggio,N,6,Pesa le tue valigie prima del viaggio per evit...,4.84
198,98,vista,N,6,è molto importante non perdere di vista la pro...,2.22


In [4]:
# Add CONCRETE column
df["CONCRETE"] = np.where(df["MEAN"] <= 4, 'LOW', 'HIGH')
df

,index,TARGET,POS,INDEX,TEXT,MEAN,CONCRETE
0,0,achievement,N,3,"Bring up academic achievements , awards , and ...",3.06,LOW
1,1,achievement,N,9,"Please list people you have helped , your pers...",3.03,LOW
2,2,activate,V,1,Add activated carbon straight to your vodka .,3.83,LOW
3,3,activate,V,15,"Place sensors around your garden , and when a ...",5.51,HIGH
4,4,adventure,N,9,Look for a partner that shares your level of a...,2.03,LOW
...,...,...,...,...,...,...,...
195,95,verità,N,8,"In un modo o nell' altro , la verità viene sem...",2.53,LOW
196,96,viaggio,N,2,Organizza dei viaggi nel fine settimana quando...,5.03,HIGH
197,97,viaggio,N,6,Pesa le tue valigie prima del viaggio per evit...,4.84,HIGH
198,98,vista,N,6,è molto importante non perdere di vista la pro...,2.22,LOW


# Question 2

In [5]:
train, test = sk.model_selection.train_test_split(df, train_size=0.8, test_size=0.1, random_state=4111)
print("Train size: ", str(len(train)), ", Test size: " + str(len(test)))

Train size:  160 , Test size: 20


# Question 3
???

In [6]:
majority = [max(train["CONCRETE"])]*len(test["CONCRETE"])

In [9]:
print(
    "\tMetrics for: Prediction using majority class\n\n",
    sk.metrics.classification_report(
        test["CONCRETE"],
        majority
    )
)
sk.metrics.confusion_matrix(test["CONCRETE"], majority)

	Metrics for: Prediction using majority class

               precision    recall  f1-score   support

        HIGH       0.00      0.00      0.00        12
         LOW       0.40      1.00      0.57         8

    accuracy                           0.40        20
   macro avg       0.20      0.50      0.29        20
weighted avg       0.16      0.40      0.23        20



array([[ 0, 12],
       [ 0,  8]])

# Question 4

In [8]:
target_length = lambda sent: "HIGH" if len(sent) >= 5 else "LOW"

In [14]:
print(
    "\tMetrics for: Prediction using majority class\n\n",
    sk.metrics.classification_report(
        test["CONCRETE"],
        [target_length(sent) for sent in test["TARGET"]]
    )
)
sk.metrics.confusion_matrix(test["CONCRETE"], [target_length(sent) for sent in test["TARGET"]])

	Metrics for: Prediction using majority class

               precision    recall  f1-score   support

        HIGH       0.54      0.58      0.56        12
         LOW       0.29      0.25      0.27         8

    accuracy                           0.45        20
   macro avg       0.41      0.42      0.41        20
weighted avg       0.44      0.45      0.44        20



array([[7, 5],
       [6, 2]])

# Question 5